<div style="text-align: center;">
    <h1>Localisation des communes </h1>
</div>


Le but de ce notebook est de télécharger la localisation de toutes les communes de France.

Pour cela on va partir de ce dataset proposé par la Poste : [Base officielle des codes postaux](https://datanova.laposte.fr/datasets/laposte-hexasmal).

Le fichier a le schéma de données suivant : 


| name                    | title                       | type    | Descriptin|
|-------------------------|-----------------------------|---------|---------------------------------------------|
| code_commune_insee      | Code commune INSEE          | string  | Code INSEE de la commune, sur 5 caractères  |
| nom_de_la_commune       | Nom de la commune            | string  | Nom d'une commune                  |
| code_postal             | Code postal                 | string  | Code Postal                 |
| libelle_d_acheminement  | Libellé d'acheminement       | string  |                                             |
| ligne_5                 | Ligne 5                     | string  |                                             |
| **contours_commune.geometry** | geometry                   | Texte   | Géométrie GeoJSON ou WKT                    | Une géométrie (point, polygone, ligne, etc.) au format GeoJSON ou WKT. |


Son intérêt réside dans la dernière ligne : certains autres datasets localisent une ville avec un centroïde calculé comme le centre géographique de la ville.

Ici, le fait d'avoir la géométrie exacte des villes permettra d'avoir une jointure plus fine avec la zone de chalandise : dès qu'une ville a un point a l'intérieur de la zone de chalandise, on considère que les entreprises qui s'y truvent et l'entièreté de sa population est décomptée.




Ces informations seront ensuite ajoutées au fichier "../Données nationales/populationCommune.parquet" constitué dans [ce notebook](../Notebooks/5.%20Localisation%20des%20communes.ipynb) et qui contient les populations par commune.

On veut garder les informations suivantes :

- codeInseeCommune
- nomCommune
- populationCommune
- geometryCommune

In [98]:
! pip install -q geopandas geodatasets python-Levenshtein


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [99]:
import os 
import pandas as pd 
import folium
import geopandas as gpd
import folium
from shapely.geometry import shape
import Levenshtein as lev


# 1. Téléchargement des données

In [3]:
os.system("wget -q -P '../Données nationales/' https://datanova.laposte.fr/data-fair/api/v1/datasets/laposte-hexasmal/data-files/019HexaSmal-full.csv")

0

In [65]:
path = "../Données nationales/019HexaSmal-full.csv"

# lecture du fichier csv avec geopandas
df = gpd.read_file(path, sep=",",geometry="_contours_commune.geometry")

df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 39193 entries, 0 to 39192
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   #Code_commune_INSEE         39193 non-null  object  
 1   Nom_de_la_commune           39193 non-null  object  
 2   Code_postal                 39193 non-null  object  
 3   Libellé_d_acheminement      39193 non-null  object  
 4   Ligne_5                     39193 non-null  object  
 5   _contours_commune.geometry  39193 non-null  object  
 6   geometry                    0 non-null      geometry
dtypes: geometry(1), object(6)
memory usage: 2.1+ MB


# 2. Nettoyage des données

A quoi correspondent les villes dont l'encodage est manquant ?

In [66]:
df[df["_contours_commune.geometry"] == ""]

,#Code_commune_INSEE,Nom_de_la_commune,Code_postal,Libellé_d_acheminement,Ligne_5,_contours_commune.geometry,geometry
10488,27058,LES TROIS LACS,27700,LES TROIS LACS,BERNIERES SUR SEINE,,None
10489,27058,LES TROIS LACS,27700,LES TROIS LACS,TOSNY,,None
10490,27058,LES TROIS LACS,27940,LES TROIS LACS,VENABLES,,None
38903,97501,MIQUELON LANGLADE,97500,ST PIERRE ET MIQUELON,LANGLADE,,None
38904,97501,MIQUELON LANGLADE,97500,ST PIERRE ET MIQUELON,MIQUELON LANGLADE,,None
38905,97502,ST PIERRE,97500,ST PIERRE ET MIQUELON,ST PIERRE,,None
38929,97701,ST BARTHELEMY,97133,ST BARTHELEMY,,,None
38930,97801,ST MARTIN,97150,ST MARTIN,,,None
38931,98611,ALO,98610,ALO,,,None
38932,98612,SIGAVE,98620,SIGAVE,,,None


Au vu du faible nombre de cas, on  les supprime

In [67]:
# drop des lignes avec des valeurs manquantes
df = df[df["_contours_commune.geometry"] != ""]

Les données sont à la maille des lignes d'acheminements, donc il y a plusieurs lignes par commune.
On va supprimer les doublons

In [68]:
# est ce qu'il y a des doublons Code_commune_INSEE qui n'ont pas la même géométrie ?
df[df.duplicated(subset=["#Code_commune_INSEE"], keep=False)].sort_values(by="#Code_commune_INSEE")

# drop des doublon
df = df.drop_duplicates(subset=["#Code_commune_INSEE"], keep="first")

Visiblement la colonne d'encodage des villes n'est pas reconnue par geopandas, on va donc forcer la reconnaissance.

In [69]:
def eval_geometry(x):
    
    try :
        return shape(eval(x))
    except Exception as e:
        return None


df["geometry"] = df["_contours_commune.geometry"].apply(lambda x: eval_geometry(x))

# 3. Affichage d'exemples aléatoires pour vérifier le fichier

In [89]:
# ville aléatoire
sample = df.sample(1)

# projection de la ville aléatoire
sample.crs = "EPSG:4326"

# centre de la france pour créer la carte
centre_france = [46.5, 2]

# affichage de la ville aléatoire dans une carte folium
m = folium.Map(location=centre_france, zoom_start=6)

# affichage du polygone de la ville aléatoire
folium.GeoJson(sample["geometry"]).add_to(m)

# zoom sur la ville aléatoire
m.fit_bounds(m.get_bounds())

# titre de la carte
title = "Carte de la commune de {} - {}".format(sample["Nom_de_la_commune"].values[0], sample["Code_postal"].values[0])

title_html = '''
             <h3 align="center" style="font-size:20px"><b>{}</b></h3>
             '''.format(title)

m.get_root().html.add_child(folium.Element(title_html))

# affichage de la carte
m



# 4. Jointure avec le fichier de population



In [90]:
path_population = "../Données nationales/populationCommune.parquet"

# lecture du fichier parquet
df_population = pd.read_parquet(path_population)

In [92]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35447 entries, 0 to 35446
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   com_arm_code     35447 non-null  object
 1   com_arm_name     35447 non-null  object
 2   com_arm_pop_mun  35447 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 830.9+ KB


Est ce que les codes INSEE sont bien raccords ?

In [95]:
# nombre de code commune INSEE df_population dans df
nb = df_population["com_arm_code"].isin(df["#Code_commune_INSEE"]).sum()
print("Nombre de code commune INSEE dans df_population et df : {} soit {:.2f}% des donnés de df_population".format(nb, nb/len(df_population)*100))

# nombre de code commune INSEE df dans df_population
nb = df["#Code_commune_INSEE"].isin(df_population["com_arm_code"]).sum()
print("Nombre de code commune INSEE dans df et df_population : {} soit {:.2f}% des donnés de df".format(nb, nb/len(df)*100))

Nombre de code commune INSEE dans df_population et df : 34969 soit 98.65% des donnés de df_population
Nombre de code commune INSEE dans df et df_population : 34969 soit 99.72% des donnés de df


Oui ! On peut donc faire une jointure.

In [96]:
# jointure entre df et df_population sur le code commune INSEE
df_join = df[['#Code_commune_INSEE', 'Nom_de_la_commune', 'Code_postal', 'geometry']].merge(df_population, left_on="#Code_commune_INSEE", right_on="com_arm_code", how="inner")

In [97]:
df_join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34969 entries, 0 to 34968
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   #Code_commune_INSEE  34969 non-null  object  
 1   Nom_de_la_commune    34969 non-null  object  
 2   Code_postal          34969 non-null  object  
 3   geometry             34969 non-null  geometry
 4   com_arm_code         34969 non-null  object  
 5   com_arm_name         34969 non-null  object  
 6   com_arm_pop_mun      34969 non-null  int64   
dtypes: geometry(1), int64(1), object(5)
memory usage: 1.9+ MB


Par acquis de conscience on vérifie que les noms correspondent avec une distance de Levenshtein

In [105]:
# distance de Levenshtein

df_join["distance"] = df_join.apply(lambda x: lev.distance(x["Nom_de_la_commune"].capitalize(), x["com_arm_name"].capitalize()), axis=1)

df_join["distance"].describe()

count    34969.000000
mean         1.398410
std          2.035218
min          0.000000
25%          0.000000
50%          0.000000
75%          2.000000
max         24.000000
Name: distance, dtype: float64

A priori pas d'inquiétude on va regarder les top

In [107]:
# top 20 des plus grosses distances
df_join.sort_values(by="distance", ascending=False).head(20)

,#Code_commune_INSEE,Nom_de_la_commune,Code_postal,geometry,com_arm_code,com_arm_name,com_arm_pop_mun,distance
5239,16052,BORS DE MONTMOREAU,16190,"POLYGON ((0.18248 45.35280, 0.18116 45.35299, ...",16052,Bors (Canton de Tude-et-Lavalette),235,24
9921,28012,VALD YERRE,28290,"POLYGON ((1.16705 48.02634, 1.16952 48.02657, ...",28012,Commune nouvelle d'Arrou,3580,20
1409,03223,ST CHRISTOPHE EN BOURBONNAIS,03120,"POLYGON ((3.62394 46.19548, 3.62493 46.19601, ...",03223,Saint-Christophe,437,19
25042,64181,CASTILLON D ARTHEZ,64370,"POLYGON ((-0.58236 43.44887, -0.58046 43.44891...",64181,Castillon (Canton d'Arthez-de-Béarn),335,19
4407,13201,MARSEILLE 01,13001,"POLYGON ((5.37214 43.29097, 5.38232 43.29348, ...",13201,Marseille 1er Arrondissement,39265,18
27256,69381,LYON 01,69001,"POLYGON ((4.83976 45.76627, 4.83957 45.77258, ...",69381,Lyon 1er Arrondissement,29303,18
29274,75101,PARIS 01,75001,"POLYGON ((2.34463 48.85409, 2.34588 48.85531, ...",75101,Paris 1er Arrondissement,16030,18
5240,16053,BORS DE BAIGNES,16360,"POLYGON ((-0.21474 45.34124, -0.21862 45.33852...",16053,Bors (Canton de Charente-Sud),120,17
29277,75104,PARIS 04,75004,"POLYGON ((2.36849 48.85581, 2.36494 48.85631, ...",75104,Paris 4e Arrondissement,29064,17
29278,75105,PARIS 05,75005,"POLYGON ((2.33666 48.83967, 2.34007 48.83878, ...",75105,Paris 5e Arrondissement,57380,17


Vendu ! On peut sauvegarder le fichier

# 5. Sauvegarde du fichier 

In [110]:
# filtre des colonnes

df_join = df_join[[ "com_arm_name", "com_arm_code", "com_arm_pop_mun", "geometry"]]

# renommage des colonnes
new_columns = ["nomCommune", "codeCommune", "populationCommune", "geometryCommune"]

df_join.columns = new_columns

# export en parquet
df_join.to_parquet("../Données nationales/populationcommunes.parquet")